In [1]:
import json
import requests
import pytse_client as tse
import pandas as pd
import matplotlib.pyplot as plt
from finta import TA
from datetime import datetime

## Features on which we choose our stocks:

### 1_Fundemental Analysis:(Filter)

•	Base volumne < mean(last week)

•   last day difference > mean(last week)+ 20% 

•	Smart money entrance

In [2]:
def get_change(current, previous)->int:
    '''
    Gets the change percentage  between 2 values
    '''
    if current == previous:
        return 100.0
    try:
        return int(((current - previous) / previous) * 100)
    except ZeroDivisionError:
        return 0


def check_vol(base_vol,ticker,verbose=True)->int:
    
    '''
    Checking if Last day Volume more than Base and mean vol
    and Calculate those Diffrences Based on that will return a Stock
    returning a score value either 1/0
    '''
    
    if base_vol < ticker['volume'].tail().mean() and ticker.iloc[-1]['volume']>ticker['volume'].tail().mean():

        #Calculating Diffenrence Percentage between Mean and Last Day Volume
        change_perc=get_change(ticker.iloc[-1]['volume'],ticker['volume'].tail().mean())
        
        #Printing extra information
        if verbose:
            print(f"""Mean: {ticker['volume'].tail().mean()}
Last: {ticker.iloc[-1]['volume']}
Difference:{change_perc}%
Highest Volume:{ticker['volume'].tail().max()}
Lowest Volume:{ticker['volume'].tail().min()}""","\n"*10)
        
        #If the volume of the last day was more than the mean of the  past week
        if change_perc >=20:
            return 1
        else:
            return 0
    else:
        
        if verbose:
            print("Stock Volume is lower than mean of the last week")
        return 0

    
def Trend_Estimation(ticker,verbose=True)->int:
    '''
    Estimates the relative trend of a stock
    and Returns a score value either 1/0
    '''
    
    #Trend Esmtimation Based on last 2 weeks
    if not ticker['adjClose'].tail(10).iloc[0] < ticker['adjClose'].tail(10).iloc[-1]:
        
        if verbose:
            print("Detected Downard Trend")
        return 1
    
    else:
        
        if verbose:
            print("Detected Upward Trend")
        return 0
        

### 2_Technical Analysis:

•	(RSI, MACD, STOCH,ICHIMOKU)

In [4]:
#Adding Custom Oscillators to a Dataframe
def _get_indicator_data(data,INDICATORS=['RSI', 'MACD', 'STOCH','STOCHD','ICHIMOKU']):
    """
    Function uses the finta API to calculate technical indicators used as the features
    :return:
    """

    for indicator in INDICATORS:
        ind_data = eval('TA.' + indicator + '(data)')
        if not isinstance(ind_data, pd.DataFrame):
            ind_data = ind_data.to_frame()
        data = data.merge(ind_data, left_index=True, right_index=True)
    data.rename(columns={"14 period EMV.": '14 period EMV'}, inplace=True)

    return data




def Buy_Sell_Index_Score(df,verbose=True)->int:
    '''
    Calculates the Technical Score of a stock
    worst score -5 and the best would be +5
    '''
    score=0
    
   
    ####################      RSI      ###################
    
    if not df['RSI'].tail().iloc[0] < df['RSI'].tail().iloc[-1]: #Downward RSI
        score+=1
        
        if int(df['RSI'].tail().iloc[-1]) in range(10,40):
            score+=1
    
    else:
        score-=1
        
        
    #Grabbing the past 2 Weeks RSI 
    RSI_LTW=df["RSI"][-10:]
    
    #checking if the mean of the last day is more than  the last 3 and 7 days 
    if  RSI_LTW.iloc[-3:].mean() < RSI_LTW.iloc[-1] and RSI_LTW.iloc[:-1].mean() < RSI_LTW.iloc[-1]:   
        score+=1
    
    else:
        score-=1   

    #####################      MACD     ####################
    
    MACD_LTWs=df["MACD-diff"][-10:]
    
    if MACD_LTWs.mean() <= -20:
        score+=2
    elif -1 > MACD_LTWs.mean() >= -5:
        score+=1
    elif 10>= MACD_LTWs.mean() >=0 :
        score-=1
    elif 20>= MACD_LTWs.mean() >=10 :
        score-=2
        
    else:
        pass
    
    
    
    #####################      STOCH     ####################
    
    STOCH_LTWs=df["STOCH-diff"][-10:]
    
    if  STOCH_LTWs.mean() <= -20:
        score+=2
    elif -1 > STOCH_LTWs.mean() >= -5:
        score+=1
    elif 10>= STOCH_LTWs.mean() >=0 :
        score-=1
    elif 20>= STOCH_LTWs.mean() >=10 :
        score-=2
        
    else:
        pass
    
    #####################     ICHIMOKU   ####################
    
    ICHI_LTWs=df["ICHI-diff"][-10:]
    
    if  ICHI_LTWs.mean() <= -20:
        score+=2
    elif -1 > ICHI_LTWs.mean() > -20:
        score+=1
    elif 10>= ICHI_LTWs.mean() >=0 :
        score-=1
    elif 20>= ICHI_LTWs.mean() >=10 :
        score-=2
        
    else:
        pass
    
    
    return score

### Downloading and Processing the Data

In [5]:
address="https://raw.githubusercontent.com/Glyphack/pytse-client/master/pytse_client/data/symbols_name.json"
response=requests.get(address)
if response.status_code ==200:
    symbols_json=json.loads(response.text)
    symbols=[ i for i in symbols_json.keys()]
    print(len(symbols)," number of symbols loaded")
else:
    print("Response failed check the Internet")

1201  number of symbols loaded


### Applying the above logic on all Stocks

In [6]:
def main(symbols:list,highest_score=5,verbose=True,write_to_file=True)->dict:
    '''
    input: list of symbols
    output:dict of symbol's fundamental and technical score + dict of highest scored stocks
    '''
    start=datetime.now()
    print("Proccess started")
    print("="*20)
    sybmol_scores={}
    Best_Stocks={}
    
    for symbol in symbols:
        try:
            #Phase 1: Downloading the Stock data
            if verbose:
                print("Fetching",symbol)
                
            ticker=tse.download(symbols=f"{symbol}", write_to_csv=True)[f"{symbol}"]
            ticker.drop(['value','count','yesterday'],axis=1,inplace=True)
            ticker.dropna(inplace= False)
            base_vol = tse.Ticker(f"{symbol}").base_volume


            #Phase 2: analysis

            #1_Fundemental

            vol_score=check_vol(base_vol,ticker,verbose=False)
            Trend_score=Trend_Estimation(ticker,verbose=False)


            #2_Technical

            ######  Preprations  ######

            #Defining Oscillators
            INDICATORS = ['RSI', 'MACD', 'STOCH','STOCHD','ICHIMOKU']


            #Adding Indicators to the DF
            df=_get_indicator_data(ticker,INDICATORS)

            #Calculating Ocillator Diffrences
            df['MACD-diff']=df['MACD'] -  df['SIGNAL']
            df['STOCH-diff']=df['14 period STOCH %K'] - df['3 period STOCH %D.']
            df['ICHI-diff'] =df['KIJUN'] - df['TENKAN']


            #New df only shows Oscillator
            OSC_df=df

            #Removing Extra Cols
            df.rename(columns={'14 period RSI':'RSI'},inplace=True)
            cl_names=list(df.columns[:-3])
            cl_names.remove('adjClose')
            cl_names.remove('RSI')
            cl_names.remove('date')
            OSC_df.index.name='Date'
            OSC_df.drop(cl_names,axis=1,inplace=True)
            #OSC_df.tail(15)

            Technical_Score=Buy_Sell_Index_Score(OSC_df,verbose=False)

            sybmol_scores[symbol]={"Fundamental":vol_score+Trend_score,"Technical":Technical_Score}

            stock_score=vol_score+Trend_score+Technical_Score

            if stock_score >= highest_score:
                Best_Stocks[symbol]=stock_score
                
                if verbose:
                    print(sybmol_scores[symbol],"\nScore sum:",stock_score)


        except Exception as e:
            if verbose:
                print(e)
                print("Passing",symbol)
            pass
    
    if write_to_file:
        
        with open("best_stocks.txt",'a',encoding='utf-8') as f:
            
            #Writing all stocks with thier scores to a file
            f.write("All Stocks")
            f.write("\n")
            f.write("="*20)
            f.write("\n")
            for k,v in sybmol_scores.items():
                f.write(f"{k}==>{v}")
                f.write("\n")
            
            
            
            #Writing Best stocks with thier scores to a file
            f.write("\n")
            f.write("Best Stocks")
            f.write("\n")
            f.write("="*20)
            f.write("\n")
            for k,v in Best_Stocks.items():
                f.write(f"{k}==>{v}")
                f.write("\n")
                
                
    ETA=datetime.now() - start
    print("="*20)
    print("Prossess is finished in ",ETA.seconds , "seconds")
    return sybmol_scores,Best_Stocks

if __name__ == "__main__":
    all_stocks,best_stocks=main(symbols=symbols,highest_score=5, verbose=True, write_to_file=True)
    print(best_stocks)